In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

import os


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import pickle
import re
import zipfile

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
#Define the path to store NLTK data
nltk_data_path = '/kaggle/working/nltk_data'

#Create Directory if it does'nt exists
if not os.path.exists(nltk_data_path):
    os.makedirs(nltk_data_path)

#Append the custom path to NLTK data paths
nltk.data.path.append(nltk_data_path)

#List the required NLTK packages
required_packages = [
    'wordnet','omw-1.4','stopwords','punkt','average_perceptron_tagger','maxent_ne_chunker','words','vader_lexicon','punkt-tab'
]

#Download all required packages 
for package in required_packages :
    nltk.download(package,download_dir=nltk_data_path)

# Function to unzip a file
def unzip_file(zip_path, extract_path):
    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Extracted {zip_path} to {extract_path}")
    else:
        print(f"File not found: {zip_path}")

# Unzip downloaded WordNet and OMW files if they exist
unzip_file(os.path.join(nltk_data_path, 'corpora', 'wordnet.zip'), os.path.join(nltk_data_path, 'corpora'))
unzip_file(os.path.join(nltk_data_path, 'corpora', 'omw-1.4.zip'), os.path.join(nltk_data_path, 'corpora'))

# Test if resources are loaded
try:
    from nltk.corpus import wordnet
    print("WordNet successfully loaded!")
    from nltk.corpus import omw
    print("OMW successfully loaded!")
except Exception as e:
    print(f"Error loading resources: {e}")


[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /kaggle/working/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading average_perceptron_tagger: Package
[nltk_data]     'average_perceptron_tagger' not found in index
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /kaggle/working/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon 

In [25]:
df = pd.read_json('/kaggle/input/ecommerce-faq-chatbot-dataset/Ecommerce_FAQ_Chatbot_dataset.json')

In [26]:
df

,questions
0,"{'question': 'How can I create an account?', 'answer': 'To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.'}"
1,"{'question': 'What payment methods do you accept?', 'answer': 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.'}"
2,"{'question': 'How can I track my order?', 'answer': 'You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.'}"
3,"{'question': 'What is your return policy?', 'answer': 'Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.'}"
4,"{'question': 'Can I cancel my order?', 'answer': 'You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process.'}"
...,...
74,"{'question': 'Can I order a product if it is listed as 'sold out' but available for pre-order?', 'answer': 'If a product is listed as 'sold out' but available for pre-order, you can place an order to secure your item. The product will be shipped once it becomes available.'}"
75,"{'question': 'Can I return a product if it was purchased with a promotional gift card?', 'answer': 'Yes, you can return a product purchased with a promotional gift card. The refund will be issued in the form of store credit or a new gift card.'}"
76,"{'question': 'Can I request a product if it is not currently available in my preferred color?', 'answer': 'If a product is not available in your preferred color, it may be temporarily out of stock. Please check back later or sign up for color notifications.'}"
77,"{'question': 'Can I order a product if it is listed as 'coming soon' and not available for pre-order?', 'answer': 'If a product is listed as 'coming soon' but not available for pre-order, you will need to wait until it is officially released and becomes available for purchase.'}"


In [27]:
pd.reset_option("all")

/tmp/ipykernel_30/3365680546.py:1: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option("all")
/tmp/ipykernel_30/3365680546.py:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option("all")


In [28]:
pd.set_option('display.max_rows',100)

In [29]:
pd.set_option('display.max_colwidth',800)

In [30]:
pd.reset_option('max_rows')

In [31]:
df

,questions
0,"{'question': 'How can I create an account?', 'answer': 'To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.'}"
1,"{'question': 'What payment methods do you accept?', 'answer': 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.'}"
2,"{'question': 'How can I track my order?', 'answer': 'You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.'}"
3,"{'question': 'What is your return policy?', 'answer': 'Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.'}"
4,"{'question': 'Can I cancel my order?', 'answer': 'You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process.'}"
...,...
74,"{'question': 'Can I order a product if it is listed as 'sold out' but available for pre-order?', 'answer': 'If a product is listed as 'sold out' but available for pre-order, you can place an order to secure your item. The product will be shipped once it becomes available.'}"
75,"{'question': 'Can I return a product if it was purchased with a promotional gift card?', 'answer': 'Yes, you can return a product purchased with a promotional gift card. The refund will be issued in the form of store credit or a new gift card.'}"
76,"{'question': 'Can I request a product if it is not currently available in my preferred color?', 'answer': 'If a product is not available in your preferred color, it may be temporarily out of stock. Please check back later or sign up for color notifications.'}"
77,"{'question': 'Can I order a product if it is listed as 'coming soon' and not available for pre-order?', 'answer': 'If a product is listed as 'coming soon' but not available for pre-order, you will need to wait until it is officially released and becomes available for purchase.'}"


In [32]:
data = pd.json_normalize(df['questions'])

In [33]:
data

,question,answer
0,How can I create an account?,"To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and PayPal as payment methods for online orders."
2,How can I track my order?,"You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment."
3,What is your return policy?,"Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions."
4,Can I cancel my order?,"You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process."
...,...,...
74,Can I order a product if it is listed as 'sold out' but available for pre-order?,"If a product is listed as 'sold out' but available for pre-order, you can place an order to secure your item. The product will be shipped once it becomes available."
75,Can I return a product if it was purchased with a promotional gift card?,"Yes, you can return a product purchased with a promotional gift card. The refund will be issued in the form of store credit or a new gift card."
76,Can I request a product if it is not currently available in my preferred color?,"If a product is not available in your preferred color, it may be temporarily out of stock. Please check back later or sign up for color notifications."
77,Can I order a product if it is listed as 'coming soon' and not available for pre-order?,"If a product is listed as 'coming soon' but not available for pre-order, you will need to wait until it is officially released and becomes available for purchase."


In [34]:
data.dtypes

question    object
answer      object
dtype: object

In [35]:
data

,question,answer
0,How can I create an account?,"To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and PayPal as payment methods for online orders."
2,How can I track my order?,"You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment."
3,What is your return policy?,"Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions."
4,Can I cancel my order?,"You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process."
...,...,...
74,Can I order a product if it is listed as 'sold out' but available for pre-order?,"If a product is listed as 'sold out' but available for pre-order, you can place an order to secure your item. The product will be shipped once it becomes available."
75,Can I return a product if it was purchased with a promotional gift card?,"Yes, you can return a product purchased with a promotional gift card. The refund will be issued in the form of store credit or a new gift card."
76,Can I request a product if it is not currently available in my preferred color?,"If a product is not available in your preferred color, it may be temporarily out of stock. Please check back later or sign up for color notifications."
77,Can I order a product if it is listed as 'coming soon' and not available for pre-order?,"If a product is listed as 'coming soon' but not available for pre-order, you will need to wait until it is officially released and becomes available for purchase."


In [36]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

nltk.data.path.append('/kaggle/working/nltk_data')

def clean_text_la(text) :
    la = WordNetLemmatizer()
    faq = re.sub(r'[^A-Za-z0-9]',' ',text)
    faq = faq.lower().split()
    faq = [la.lemmatize(word) for word in faq if not word in stopwords.words('english')]
    return ' '.join(faq)

In [37]:
# Apply cleaning function to both questions columns
data['question']= data['question'].apply(clean_text_la)

In [38]:
#defining preprocessing function for answer
def preprocess_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s,.`']","",text)
    text = re.sub(r"\s+"," ",text).strip()
    text = text.lower()
    return text

# Apply cleaning function to both answer columns    
data['answer']= data['answer'].apply(preprocess_text)

In [39]:
data

,question,answer
0,create account,"to create an account, click on the 'sign up' button on the top right corner of our website and follow the instructions to complete the registration process."
1,payment method accept,"we accept major credit cards, debit cards, and paypal as payment methods for online orders."
2,track order,"you can track your order by logging into your account and navigating to the 'order history' section. there, you will find the tracking information for your shipment."
3,return policy,"our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. please refer to our returns page for detailed instructions."
4,cancel order,"you can cancel your order if it has not been shipped yet. please contact our customer support team with your order details, and we will assist you with the cancellation process."
...,...,...
74,order product listed sold available pre order,"if a product is listed as 'sold out' but available for preorder, you can place an order to secure your item. the product will be shipped once it becomes available."
75,return product purchased promotional gift card,"yes, you can return a product purchased with a promotional gift card. the refund will be issued in the form of store credit or a new gift card."
76,request product currently available preferred color,"if a product is not available in your preferred color, it may be temporarily out of stock. please check back later or sign up for color notifications."
77,order product listed coming soon available pre order,"if a product is listed as 'coming soon' but not available for preorder, you will need to wait until it is officially released and becomes available for purchase."


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [41]:
X = vectorizer.fit_transform(data['question'])

In [42]:
X.shape

(79, 119)

In [43]:
from sklearn.model_selection import train_test_split
   
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, data['answer'], test_size=0.2, random_state=42)

In [44]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [47]:
def get_response(query) :
    query_clean = clean_text_la(query)
    query_vector=vectorizer.transform([query_clean])
    similarity_scores=cosine_similarity(query_vector,X)
    most_similar_idx = np.argmax(similarity_scores)
    response = data['answer'].iloc[most_similar_idx]
    print(f"Question : {data['question'].iloc[most_similar_idx]}")
    print(f"Answer : {response}")

In [48]:
get_response("return policy ?")

Question : return policy
Answer : our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. please refer to our returns page for detailed instructions.
